In [1]:
import sys
sys.path.append('/Users/vandanchopra/Vandan_Personal_Folder/CODE_STUFF/Projects/mathematricks/')
import os
import pickle
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import plotly.graph_objects as go
import plotly.express as px
from pathlib import Path
import yfinance as yf
import time
from datetime import timedelta
from systems.performance_reporter import StrategyReporter, PerformanceReporter
from systems.utils import MarketDataExtractor

In [2]:
market_data_extractor = MarketDataExtractor()
strategy_reporter = StrategyReporter()
performance_reporter = PerformanceReporter(market_data_extractor)

In [3]:
# All the Inputs for the Backtest that needs to be analyzed
# testname = 'e69c9c193856ccf322ef0db21cb59676841f63def2a5dd9ccd3ab3d69960c423' # Profits not Reinvested
testname = '7f18b860646c324b6b8f2a67e2b87e307a19e4fccc375a8fbc71fdaf3ec4f38a' # Profits Reinvested
save_folder = "/Users/vandanchopra/Vandan_Personal_Folder/CODE_STUFF/Projects/mathematricks/db/data/yahoo/1d"
test_folder_path = f'/Users/vandanchopra/Vandan_Personal_Folder/CODE_STUFF/Projects/mathematricks/db/vault/backtest_reports/{testname}'
backtest_output = strategy_reporter.load_backtest_output(test_folder_path)
open_orders = backtest_output['open_orders']
closed_orders = backtest_output['closed_orders']
config_dict = backtest_output['config_dict']

In [4]:
#Processing Open order to make them close, and remove history
open_orders_list = strategy_reporter.clean_up_order_list(open_orders, drop_history=True)
#Processing Closed orders to remove history
closed_orders_list = strategy_reporter.clean_up_order_list(closed_orders, drop_history=True)
#Merge both orders
final_list = open_orders_list + closed_orders_list
# Step 2: Sort the filtered orders by exit order `filled_timestamp`
sorted_orders = sorted(final_list, key=lambda x: x[1]['filled_timestamp'])
# Step 3: Fetch the first and last `filled_timestamp` with buffer
start_date = sorted_orders[0][1]['filled_timestamp'] - timedelta(days=365)
end_date = sorted_orders[-1][1]['filled_timestamp'] + timedelta(days=365)
# Current we are downloading csv file for below three index
indices = ["^DJI", "^IXIC", "^GSPC"]
index_data_dict = strategy_reporter.load_and_save_index_data(indices, start_date, end_date, save_folder)
rolling_window = int(max(60, (end_date - start_date).days / 20))
print({f"Analyzing with a rolling window of {rolling_window} days"})

{'Analyzing with a rolling window of 400 days'}


### This below function calculates profit/loss from the orders  **(exitPrice - entryPrice) * orderQuantity**

This function also plot the progress lines for 3 indexes provided earlier.

In [5]:
# Example usage
starting_capital = 72181
trade_df_by_date = strategy_reporter.create_data_input_for_cumulative_returns_and_indices(final_list, index_data_dict, starting_capital)
strategy_reporter.plot_cumulative_returns_and_indices(trade_df_by_date, index_data_dict)


# Plot Rolling charts

#### Sharpe Ratio

In [ ]:
index_data_dict_ = index_data_dict.copy()
trade_df_by_date_ = trade_df_by_date.copy()
rolling_sharpe_df = strategy_reporter.calculate_rolling_sharpe_ratio(trade_df_by_date_, index_data_dict_, index_name='^IXIC', window=rolling_window)
# rolling_sharpe_df.plot(title='Rolling Sharpe Ratio', figsize=(35, 5), grid=True)
strategy_reporter.plot_rolling_sharpe_ratio(rolling_sharpe_df, index_data_dict)

### Rolling Win Percentage

In [ ]:
trade_df_by_order = strategy_reporter.prepare_trade_data_df_by_order(final_list)
strategy_reporter.plot_rolling_win_percentage(trade_df_by_order, index_data_dict, window=rolling_window)

### Rolling Average Profit and Loss



***Note***

**Average Profit=Sum of all profits/Number of profitable trades**

**Rolling Average Profitt=Sum of profits in the last ’n’ trades up to t/Number of profitable trades in the last ’n’ trades up to t**

Rolling average is calculated using this formula

In [ ]:
def calculate_rolling_profit_loss(trade_data, rolling_window):
    # Ensure profit_loss is numeric
    trade_data['profit_loss'] = pd.to_numeric(trade_data['profit_loss'], errors='coerce').fillna(0)

    # Create temporary columns to separate profits and losses
    trade_data['profit_only'] = trade_data['profit_loss'].apply(lambda x: x if x > 0 else 0)
    trade_data['loss_only'] = trade_data['profit_loss'].apply(lambda x: x if x < 0 else 0)

    # Calculate rolling averages
    rolling_profit = trade_data['profit_only'].rolling(rolling_window).mean()
    rolling_loss = trade_data['loss_only'].rolling(rolling_window).mean()

    # Return filled Series to avoid NaNs
    return rolling_profit.fillna(0), rolling_loss.fillna(0)

def plot_rolling_profit_loss(trade_data, rolling_profit, rolling_loss):
    """
    Plot Rolling Average Profit and Loss on the same chart.

    Parameters:
        trade_data (DataFrame): Original trade data for x-axis.
        rolling_profit (Series): Rolling average profit.
        rolling_loss (Series): Rolling average loss.
    """
    fig = go.Figure()

    # Add Rolling Average Profit
    fig.add_trace(go.Scatter(
        x=trade_data['filled_timestamp'],
        y=rolling_profit,
        mode='lines',
        name='Rolling Average Profit',
        line=dict(color='green')
    ))

    # Add Rolling Average Loss
    fig.add_trace(go.Scatter(
        x=trade_data['filled_timestamp'],
        y=rolling_loss,
        mode='lines',
        name='Rolling Average Loss',
        line=dict(color='red')
    ))

    # Customize layout
    fig.update_layout(
        title="Rolling Average Profit and Loss",
        xaxis_title="Filled Timestamp",
        yaxis_title="Value",
        template="plotly_white",
        legend_title="Metric",
        hovermode="x unified"
    )

    fig.show()

# trade_trade_dfdata = prepare_trade_data(final_list)
rolling_profit, rolling_loss = calculate_rolling_profit_loss(trade_df_2, rolling_window)
plot_rolling_profit_loss(trade_df_2, rolling_profit, rolling_loss)


### Rolling Proft Factor

In [ ]:
def plot_rolling_profit_factor(trade_df, window):

    # Separate profit and loss
    trade_df['profit'] = trade_df['profit_loss'].apply(lambda x: x if x > 0 else 0)
    trade_df['loss'] = trade_df['profit_loss'].apply(lambda x: abs(x) if x < 0 else 0)

    # Compute rolling sums for profits and losses
    trade_df['rolling_profit_sum'] = trade_df['profit'].rolling(window=window).sum()
    trade_df['rolling_loss_sum'] = trade_df['loss'].rolling(window=window).sum()

    # Calculate rolling profit factor
    trade_df['rolling_profit_factor'] = trade_df['rolling_profit_sum'] / trade_df['rolling_loss_sum']
    trade_df['rolling_profit_factor'] = trade_df['rolling_profit_factor'].replace([float('inf'), float('nan')], 0)

    # Plot using Plotly
    fig = px.line(
        trade_df,
        x='filled_timestamp',
        y='rolling_profit_factor',
        title=f'Rolling Profit Factor (Window Size: {window})',
        labels={'rolling_profit_factor': 'Rolling Profit Factor', 'filled_timestamp': 'Filled Timestamp'},
        template='plotly_dark'
    )
    fig.update_layout(yaxis=dict(title='Profit Factor'))
    fig.show()

plot_rolling_profit_factor(trade_df, rolling_window)

### Rolling Profit Long and Short Trades

6. Rolling Average Profit for Long Trades¶
7. Rolling Average Profit for Short Trades
8. Rolling Average Loss for Long Trades
9. Rolling Average Loss for Short Trades

In [ ]:
def plot_rolling_avg_profit_loss(trade_df, window=10):
    # Separate long and short trades
    long_trades = trade_df[trade_df['trade_type'] == 'long'].copy()
    short_trades = trade_df[trade_df['trade_type'] == 'short'].copy()

    # Function to calculate rolling averages
    def calculate_rolling_averages(trades, window):
        trades['rolling_avg_profit'] = (
            trades['profit_loss']
            .where(trades['profit_loss'] > 0)  # Filter profits
            .rolling(window=window, min_periods=1)
            .mean()
        )
        trades['rolling_avg_loss'] = (
            trades['profit_loss']
            .where(trades['profit_loss'] < 0)  # Filter losses (negative values)
            .rolling(window=window, min_periods=1)
            .mean()
        )
        return trades

    # Calculate rolling averages
    long_trades = calculate_rolling_averages(long_trades, window)
    short_trades = calculate_rolling_averages(short_trades, window)

    # Plot for long trades
    fig_long = px.line(
        long_trades,
        x='filled_timestamp',
        y=['rolling_avg_profit', 'rolling_avg_loss'],
        title=f'Rolling Average Profit and Loss for Long Trades (Window Size: {window})',
        labels={'value': 'Rolling Average', 'exit_timestamp': 'Exit Timestamp'},
        template='plotly_dark'
    )
    fig_long.update_traces(mode='lines+markers')
    fig_long.update_layout(
        yaxis=dict(title='Average Amount'),
        legend_title="Metrics"
    )
    fig_long.show()

    # Plot for short trades
    fig_short = px.line(
        short_trades,
        x='filled_timestamp',
        y=['rolling_avg_profit', 'rolling_avg_loss'],
        title=f'Rolling Average Profit and Loss for Short Trades (Window Size: {window})',
        labels={'value': 'Rolling Average', 'filled_timestamp': 'Exit Timestamp'},
        template='plotly_dark'
    )
    fig_short.update_traces(mode='lines+markers')
    fig_short.update_layout(
        yaxis=dict(title='Average Amount'),
        legend_title="Metrics"
    )
    fig_short.show()

# Example usage:
plot_rolling_avg_profit_loss(trade_df, window=rolling_window)


### Drawdown

In [24]:
trade_df_by_date_drawdown = strategy_reporter.calculate_drawdowns(trade_df_by_date)
strategy_reporter.plot_drawdown(trade_df_by_date_drawdown, index_data_dict)

AttributeError: 'StrategyReporter' object has no attribute 'calculate_drawdowns'

In [ ]:

# Example usage


np.float64(-12.279176992233898)

#### Long short Exposure

In [ ]:
def plot_long_short_ratio(data, freq='D'):
    exposures = []

    # Iterate over trades and calculate exposures
    for trade in data:
        entry = trade[0]

        # Determine direction and notional value of the trade
        direction = entry['orderDirection']
        quantity = entry['orderQuantity']
        entry_price = entry['fill_price']
        notional_value = quantity * entry_price

        # Add exposure with timestamp
        exposures.append({
            'Timestamp': pd.to_datetime(entry['filled_timestamp']),
            'Exposure': notional_value if direction == 'BUY' else -notional_value
        })

    # Create a DataFrame
    df = pd.DataFrame(exposures)

    # Sort by Timestamp and aggregate exposures by the specified frequency
    df = df.set_index('Timestamp').resample(freq).sum().reset_index()

    # Calculate cumulative long and short exposures
    df['LongExposure'] = df['Exposure'].where(df['Exposure'] > 0, 0).cumsum()
    df['ShortExposure'] = df['Exposure'].where(df['Exposure'] < 0, 0).cumsum()

    # Calculate long and short ratios
    df['LongRatio'] = abs(df['LongExposure']) / (abs(df['LongExposure']) + abs(df['ShortExposure']))
    df['ShortRatio'] = abs(df['ShortExposure']) / (abs(df['LongExposure']) + abs(df['ShortExposure']))

    # Plot using Plotly
    fig = go.Figure()

    # Add long ratio trace
    fig.add_trace(go.Scatter(
        x=df['Timestamp'],
        y=df['LongRatio'],
        mode='lines',
        name='Long Ratio',
        line=dict(color='green')
    ))

    # Add short ratio trace
    fig.add_trace(go.Scatter(
        x=df['Timestamp'],
        y=df['ShortRatio'],
        mode='lines',
        name='Short Ratio',
        line=dict(color='red')
    ))

    # Update layout
    fig.update_layout(
        title='Long and Short Ratio Over Time',
        xaxis_title='Time',
        yaxis_title='Ratio',
        legend_title='Ratio Type',
        template='plotly_white'
    )

    fig.show()

# Plot the chart with the sample data
plot_long_short_ratio(final_list, freq='D')


### Average Hold Time

In [ ]:

def plot_average_hold_times(data):
    hold_times = []

    # Calculate hold time and profit/loss for each trade
    for trade in data:
        entry = trade[0]
        exit_ = trade[1]

        # Calculate hold time (in days)
        hold_time = (exit_['filled_timestamp'] - entry['filled_timestamp']).total_seconds() / (24 * 3600)

        # Calculate profit/loss
        entry_value = entry['fill_price'] * entry['orderQuantity']
        exit_value = exit_['fill_price'] * exit_['orderQuantity']
        profit_loss = exit_value - entry_value

        # Determine trade type
        direction = entry['orderDirection']

        # Append to list
        hold_times.append({
            'HoldTime': hold_time,
            'ProfitLoss': profit_loss,
            'Direction': direction
        })

    # Convert to DataFrame
    df = pd.DataFrame(hold_times)

    # Calculate average hold time for each category
    avg_hold_all = df['HoldTime'].mean()
    avg_hold_long = df[df['Direction'] == 'BUY']['HoldTime'].mean()
    avg_hold_short = df[df['Direction'] == 'SELL']['HoldTime'].mean()
    avg_hold_profit = df[df['ProfitLoss'] > 0]['HoldTime'].mean()
    avg_hold_loss = df[df['ProfitLoss'] <= 0]['HoldTime'].mean()

    # Prepare data for plotting
    categories = ['All Trades', 'Long Trades', 'Short Trades', 'Profit Trades', 'Loss Trades']
    avg_hold_times = [avg_hold_all, avg_hold_long, avg_hold_short, avg_hold_profit, avg_hold_loss]

    # Plot using Plotly
    fig = go.Figure()

    # Add bar chart
    fig.add_trace(go.Bar(
        x=categories,
        y=avg_hold_times,
        marker_color=['blue', 'green', 'red', 'orange', 'purple'],
        name='Average Hold Time'
    ))

    # Update layout
    fig.update_layout(
        title='Average Hold Time by Trade Category',
        xaxis_title='Trade Category',
        yaxis_title='Average Hold Time (days)',
        template='plotly_white'
    )

    fig.show()

plot_average_hold_times(final_list)


### Take a order from closed_order and plot the price of the asset (1 year before entry to 15 days after exit) and plot the entry and exit on the graph. Also, plot the various stoplosses from the order history on the graph.
### (Make sure to have a folder_path for prices data and check that folder if the data is already there. If not, download it and save it there)
### Please see the sample image in the output folder and the code below for how that image was generated. Feel free to use the code below.

In [ ]:
def plot_order_data(order_data):
    stoploss_levels = []
    stoploss_dates = []

    # Extract order data
    entry_order = order_data[0]  # First dict: Entry order
    exit_order = order_data[1]   # Second dict: Exit order

    symbol = entry_order['symbol']
    entry_date = entry_order['filled_timestamp']
    exit_date = exit_order['filled_timestamp']
    
    entry_price = float(entry_order['fill_price'])
    exit_price = float(exit_order['fill_price'])

    # Step 3: Generate the data for the symbol
    start_date = entry_date - timedelta(days=365)  # 1 year before entry
    end_date = exit_date + timedelta(days=15)      # 15 days after exit
    start_date = start_date.replace(tzinfo=None)
    end_date = end_date.replace(tzinfo=None)
    # Step 4: Load the generated data CSV
    save_folder = '/trade_data'
    data_dict = load_and_save_index_data([symbol], start_date, end_date, save_folder)
    df = data_dict[symbol].copy()
    df['filled_timestamp'] = df.index.to_series()
    # Ensure 'Close' column is numeric and data is sorted by timestamp
    df['close'] = pd.to_numeric(df['close'], errors='coerce')
    df.dropna(subset=['close'], inplace=True)
    df = df.sort_values(by='filled_timestamp')

    # Initialize the figure with price data
    fig = px.line(df, x='filled_timestamp', y='close', title=f'{symbol} Price Chart', labels={'close': 'Price', 'filled_timestamp': 'Date'})

    # Add entry and exit markers
    fig.add_trace(go.Scatter(
        x=[entry_date], y=[entry_price],
        mode='markers+text',
        name='Entry',
        marker=dict(color='green', size=12, symbol='triangle-up'),
        text=[f"Entry: {entry_price}"],
        textposition="bottom center"
    ))

    fig.add_trace(go.Scatter(
        x=[exit_date], y=[exit_price],
        mode='markers+text',
        name='Exit',
        marker=dict(color='red', size=12, symbol='triangle-down'),
        text=[f"Exit: {exit_price}"],
        textposition="top center"
    ))
 

    # Add stop-loss levels
    if 'history' in exit_order and exit_order['history']:
        stoploss_levels = [float(stoploss_order['stoploss_abs']) for stoploss_order in exit_order['history']]
        stoploss_dates = [stoploss_order['timestamp'] for stoploss_order in exit_order['history']]

        for stoploss_date, stoploss_level in zip(stoploss_dates, stoploss_levels):
            # Add stop-loss line
            fig.add_trace(go.Scatter(
                x=[stoploss_date], y=[stoploss_level],
                mode='markers+text',
                name=f'Stop-Loss {stoploss_level}',
                marker=dict(color='orange', size=10, symbol='cross'),
                text=[f"Stoploss: {stoploss_level}"],
                textposition="top center"
            ))

    # Adjust y-axis for better visibility
    min_y = min(df['close'].min(), entry_price, exit_price, *stoploss_levels)
    max_y = max(df['close'].max(), entry_price, exit_price, *stoploss_levels)
    fig.update_yaxes(range=[min_y * 0.9, max_y * 1.1])

    fig.update_layout(
        title={
            'text': f"{symbol} Price from {start_date.date()} to {end_date.date()}",
            'x': 0.5,
            'xanchor': 'center'
        },
        xaxis_title="Date",
        yaxis_title="Price",
        legend_title="Legend",
        template="plotly_dark",
        hovermode="x unified"
    )

    # Show the graph
    fig.show()

plot_order_data(closed_orders[200])


### This takes a index and fetch one year before and one year after the first and last trade and plots the trades on the chart

# **Note Read the below comment**
## Need to make Symbols correct on this chart: Entry Long: Up Green Arrow (<span style="color: green;">▲</span>), Entry Short: Down Red Arrow 🔻, Exit Profit: Green Box 🟩 , Exit Loss: Red Box 🟥

In [ ]:
def process_and_plot_orders(data, symbol, save_folder):
    # Step 1: Filter orders for the given symbol
    filtered_orders = [entry for entry in data if entry[0]['symbol'] == symbol]
    if not filtered_orders:
        raise ValueError(f"No orders found for symbol: {symbol}")
    
    # Step 2: Sort the filtered orders by exit order `filled_timestamp`
    sorted_orders = sorted(filtered_orders, key=lambda x: x[1]['filled_timestamp'])
    
    # Convert filled_timestamp to datetime for comparison
    first_timestamp = sorted_orders[0][1]['filled_timestamp'] - timedelta(days=365)
    last_timestamp = sorted_orders[-1][1]['filled_timestamp'] + timedelta(days=365)

    # first_timestamp = start_date.replace(tzinfo=None)
    # last_timestamp = end_date.replace(tzinfo=None)
    
    # Step 4: Get index data and load it from the CSV file
    data_dict = load_and_save_index_data([symbol], first_timestamp, last_timestamp, save_folder)
    index_df = data_dict[symbol].copy()
    index_df['close'] = index_df['close'][1:].astype(float)
    
    # Step 5: Prepare data for Plotly chart
    fig = go.Figure()
    
    # Add index line
    fig.add_trace(go.Scatter(
        x=index_df.index,
        y=index_df['close'],
        mode='lines',
        name='Index'
    ))
    
    # Add entry and exit markers
    for order in sorted_orders:
        entry_order = order[0]
        exit_order = order[1]
        
        # Determine entry marker
        if entry_order['orderDirection'].upper() == 'BUY':  # Long entry
            entry_marker = dict(color='green', size=10, symbol='triangle-up')
        else:  # Short entry
            entry_marker = dict(color='red', size=10, symbol='triangle-down')
        
        # Determine exit marker based on profit/loss
        if entry_order['orderDirection'].upper() == 'BUY':  # Long trade
            trade_result = exit_order['fill_price'] - entry_order['fill_price']
        else: 
            trade_result = entry_order['fill_price'] - exit_order['fill_price']
        
        if trade_result > 0: 
            exit_marker = dict(color='green', size=10, symbol='square')
        else:  # Loss
            exit_marker = dict(color='red', size=10, symbol='square')

        # Entry marker
        fig.add_trace(go.Scatter(
            x=[entry_order['filled_timestamp']],
            y=[entry_order['fill_price']],
            mode='markers',
            marker=entry_marker,
            name='Entry Order'
        ))
        
        # Exit marker
        fig.add_trace(go.Scatter(
            x=[exit_order['filled_timestamp']],
            y=[exit_order['fill_price']],
            mode='markers',
            marker=exit_marker,
            name='Exit Order'
        ))
    
    # Set chart title and labels
    fig.update_layout(
        title=f'Order Analysis for Symbol: {symbol}',
        xaxis_title='Timestamp',
        yaxis_title='Price',
        template='plotly_dark'
    )
    
    # Display the chart
    fig.show()
    
process_and_plot_orders(final_list, 'NFLX', save_folder)

## Monthly and Weekly returns %


In [ ]:
def calculate_returns(final_list, frequency):
    trades = []

    for sublist in final_list:
        entry, exit = sublist

        if entry['orderDirection'].upper() == 'BUY':  # Long trade
            trade_return = (exit['fill_price'] - entry['fill_price']) / entry['fill_price']
        else:  # Short trade
            trade_return = (entry['fill_price'] - exit['fill_price']) / entry['fill_price']

        if frequency == 'monthly':
            trade_period = exit['filled_timestamp'].strftime('%Y-%m')
        elif frequency == 'weekly':
            trade_period = exit['filled_timestamp'].strftime('%Y-%U')
        elif frequency == 'daily':
            trade_period = exit['filled_timestamp'].strftime('%Y-%m-%d')
        else:
            raise ValueError("Frequency must be 'monthly', 'weekly', or 'daily'")

        trades.append({'period': trade_period, 'return': trade_return * 100})

    df = pd.DataFrame(trades)
    return df


def plot_heatmap(df, frequency):

    df['year'] = df['period'].str[:4]

    if frequency == 'monthly':
        df['month'] = df['period'].str[5:]
        pivot_df = (
            df.pivot_table(index='year', columns='month', values='return', aggfunc='mean')
            .fillna(0)
            .reindex(sorted(df['month'].unique(), key=lambda x: int(x)), axis=1)
        )
        x_axis_title = "Month"
        title = 'Monthly Returns Heatmap'
    elif frequency == 'weekly':
        df['week_number'] = df['period'].str[5:]
        pivot_df = (
            df.pivot_table(index='year', columns='week_number', values='return', aggfunc='mean')
            .fillna(0)
            .reindex(sorted(df['week_number'].unique(), key=lambda x: int(x)), axis=1)
        )
        x_axis_title = "Week"
        title = 'Weekly Returns Heatmap'
    else:
        raise ValueError("Frequency must be 'monthly' or 'weekly' for this function")

    max_abs_return = max(abs(pivot_df.min().min()), abs(pivot_df.max().max()))
    color_range = [-max_abs_return * 1.1, max_abs_return * 1.1]

    fig = px.imshow(
        pivot_df,
        text_auto='.3f',
        color_continuous_scale=['red', 'white', 'green'],
        labels={'color': f'{frequency.capitalize()} Return %'},
        title=title,
        zmin=color_range[0],
        zmax=color_range[1]
    )

    fig.update_layout(
        xaxis_title=x_axis_title,
        yaxis_title="Year",
        coloraxis_showscale=False,
        margin=dict(t=40, l=10, b=10, r=50),
        xaxis=dict(ticktext=pivot_df.columns.tolist()),
        yaxis=dict(type='category'),
    )

    fig.show()

In [ ]:

monthly_df = calculate_returns(final_list, 'monthly')
plot_heatmap(monthly_df, 'monthly')

weekly_df = calculate_returns(final_list, 'weekly')
plot_heatmap(weekly_df, 'weekly')

In [ ]:
def plot_daily_heatmap(final_list):

    # Calculate returns
    daily_df = calculate_returns(final_list, 'daily')
    daily_df['date'] = pd.to_datetime(daily_df['period'])
    daily_df['year'] = daily_df['date'].dt.year
    daily_df['day_of_year'] = daily_df['date'].dt.dayofyear

    # Create pivot table for heatmap
    pivot_df = daily_df.pivot_table(
        index='year', columns='day_of_year', values='return', aggfunc='mean'
    ).fillna(0)

    max_abs_return = max(abs(pivot_df.min().min()), abs(pivot_df.max().max()))
    color_range = [-max_abs_return * 1.1, max_abs_return * 1.1]

    # Plot heatmap
    fig = px.imshow(
        pivot_df,
        text_auto='.3f',
        color_continuous_scale=['red', 'white', 'green'],
        labels={'color': 'Daily Return (%)'},
        title='Daily Returns Heatmap',
        zmin=color_range[0],
        zmax=color_range[1]
    )

    fig.update_layout(
        xaxis_title="",
        yaxis_title="Year",
        coloraxis_showscale=False,
        margin=dict(t=40, l=10, b=10, r=50),
        xaxis=dict(
            tickmode='array',
            tickvals=[], 
            ticktext=[],  # Remove tick text from x-axis
            showgrid=False,
            rangeslider=dict(visible=True),  
        ),
        yaxis=dict(type='category'),
    )

    # Add custom hover data for x-axis (day of year converted to date format)
    hover_data = {
        "day_of_year": daily_df.groupby('day_of_year')['date'].first().dt.strftime('%Y-%m-%d').to_dict()
    }
    for col in pivot_df.columns:
        hover_label = hover_data["day_of_year"].get(col, col)
        fig.data[0].hovertemplate = fig.data[0].hovertemplate.replace(
            str(col), f'Day {col} ({hover_label})'
        )

    fig.show()


In [ ]:
# Plot daily returns line chart
plot_daily_heatmap(final_list)